In [64]:
import pickle
from phystables import BasicTable, SimpleTable, make_noisy
from phystables.constants import *
import define_tables
from define_tables import WallData, GoalData, BallData, TableData
import pygame as pg
import argparse
import numpy as np


DEF_NOISE = {
"kapv": KAPV_DEF, # VonMises precision on initial velocity angle estimation
"kapb": KAPB_DEF, # VonMises precision on the bounce angle estimation
"kapm": KAPM_DEF, # VonMises precision on the motion jitter
"perr": PERR_DEF  # Gaussian error on initial position estimation
}

def _ballhit(balllist):
    blcol = list(map(lambda b: b.col, balllist))
    print("Observed ball collisions: ", blcol)

def _goalhit(ball, goal):
    gtype = get_const(goal.ret)
    print("Ball", ball.col, "hit goal", gtype)

        
def getTable(tables_file, table_id, trial_id, noisy, callback=False, printTable=False):
    '''
    tables_file: File name with tables
    table_id: Table from table_files
    trial_id: Determines which simulation to run 
    noisy: T/F
    callback: T/F
    '''
    with open(tables_file, 'rb') as input:
        tablesData = pickle.load(input)
    tableData = tablesData[table_id]
    

            
    
    table = BasicTable(dims=tableData.dims)
    if printTable:
        print('\nWall Data:')
    for i, wall in enumerate(tableData.walls):
        table.add_wall(upperleft=wall.upperleft, lowerright=wall.lowerright)
        if printTable:
            print(str(i)+ ':', 'wall.upperleft', wall.upperleft)
            print(str(i)+ ':', 'wall.lowerright', wall.lowerright)

    if printTable:
            print('\nGoal Data:')
    for i, goal in enumerate(tableData.goals):
        table.add_goal(upperleft=goal.upperleft, lowerright=goal.lowerright, onreturn=goal.onreturn, color=goal.color)
        if printTable:
            print(str(i)+ ':', 'goal.upperleft', goal.upperleft)
            print(str(i)+ ':', 'goal.lowerright', goal.lowerright)
            print(str(i)+ ':', 'goal.onreturn', goal.onreturn)
            print(str(i)+ ':', 'goal.color', goal.color)
        
    balls = tableData.balls
    if trial_id == 0 or trial_id==1:
        table.add_ball(initpos=balls[trial_id].initpos, initvel=balls[trial_id].initvel, color=balls[trial_id].color)
        if printTable:
            print('\nBall Data:')
            print(str(trial_id)+ ':'+ ':', 'balls.color', balls[trial_id].color)
            print(str(trial_id)+ ':', 'balls.initpos', balls[trial_id].initpos)
            print(str(trial_id)+ ':', 'balls.initvel', balls[trial_id].initvel)
    else:
        table.add_ball(initpos=balls[0].initpos, initvel=balls[0].initvel, color=balls[0].color)
        table.add_ball(initpos=balls[1].initpos, initvel=balls[1].initvel, color=balls[1].color)
        if printTable:
            print('\nBall Data:')
            print(str(0)+ ':', 'balls.color', balls[0].color)
            print(str(0)+ ':', 'balls.initpos', balls[0].initpos)
            print(str(0)+ ':', 'balls.initvel', balls[0].initvel)
            print(str(1)+ ':', 'balls.color', balls[1].color)
            print(str(1)+ ':', 'balls.initpos', balls[1].initpos)
            print(str(1)+ ':', 'balls.initvel', balls[1].initvel)
        
    if noisy:
        table = make_noisy(table, **DEF_NOISE)
        
    if callback:
        table.on_ballhit = _ballhit
        table.on_goalhit = _goalhit
    
    return table

def run_to_end(trial, max_time=10.):
    while True:
        r = trial.step(t=max_time, maxtime=max_time)
        if r is not None:
            if isinstance(r, list):
                r = r[0]
            return [r, trial.tm]
    
def run_simulations(tables_file, table_id, trial_id, noisy, num_simulations, max_time, callback=False):    
    simulations = []
    for i in range(num_simulations):
        table = getTable(tables_file, table_id, trial_id, noisy, callback)
        ret, time = run_to_end(table, max_time)
        outcome = get_const(ret)
        ball = None
        sample = (table_id, outcome, time, ball)
        simulations.append(sample)
    return simulations

In [65]:
!python define_tables.py

pygame 2.0.0 (SDL 2.0.12, python 3.7.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Darwin (64bit) [/Users/julio/Packages/anaconda3/envs/physenv/lib/python3.7/site-packages/pymunk/libchipmunk.dylib]


In [66]:
table = getTable(tables_file='tables_metadata.pkl', 
                 table_id=0, 
                 trial_id=0, 
                 noisy=True, 
                 printTable=True)



Wall Data:
0: wall.upperleft (200, 150)
0: wall.lowerright (500, 250)
1: wall.upperleft (450, 400)
1: wall.lowerright (550, 600)
2: wall.upperleft (650, 100)
2: wall.lowerright (700, 200)
3: wall.upperleft (650, 200)
3: wall.lowerright (1000, 250)

Goal Data:
0: goal.upperleft (250, 550)
0: goal.lowerright (350, 600)
0: goal.onreturn 201
0: goal.color (255, 0, 0)
1: goal.upperleft (650, 550)
1: goal.lowerright (750, 600)
1: goal.onreturn 202
1: goal.color (0, 255, 0)

Ball Data:
0:: balls.color (160, 32, 240)
0: balls.initpos (70, 500)
0: balls.initvel (300, -300)


# Get Samples

In [67]:
for i in range(5):
    samples1 = run_simulations(tables_file='tables_metadata.pkl', 
                           table_id=0, 
                           trial_id=0, 
                           noisy=True, 
                           num_simulations=20, 
                           max_time=10)
    print()
    print(*samples1, sep = "\n")

#samples2 = run_simulations(tables_file='tables_metadata.pkl', 
#                           table_id=0, 
#                           trial_id=1, 
#                           noisy=True, 
#                           num_simulations=10, 
#                           max_time=10,
#                           callback=True)


(0, 'TIMEUP', 10.000999999999896, None)
(0, 'TIMEUP', 10.000999999999896, None)
(0, 'REDGOAL', 7.893000000000971, None)
(0, 'REDGOAL', 1.5199999999999434, None)
(0, 'GREENGOAL', 3.803999999999692, None)
(0, 'TIMEUP', 10.000999999999896, None)
(0, 'GREENGOAL', 3.0939999999997703, None)
(0, 'REDGOAL', 1.340999999999963, None)
(0, 'REDGOAL', 4.403999999999805, None)
(0, 'GREENGOAL', 2.8989999999997917, None)
(0, 'REDGOAL', 1.5179999999999436, None)
(0, 'GREENGOAL', 2.9479999999997863, None)
(0, 'GREENGOAL', 3.112999999999768, None)
(0, 'REDGOAL', 9.61700000000011, None)
(0, 'REDGOAL', 1.5389999999999413, None)
(0, 'REDGOAL', 4.0719999999996945, None)
(0, 'GREENGOAL', 3.0029999999997803, None)
(0, 'GREENGOAL', 9.349000000000258, None)
(0, 'REDGOAL', 1.3399999999999632, None)
(0, 'GREENGOAL', 3.269999999999751, None)

(0, 'GREENGOAL', 4.676999999999897, None)
(0, 'TIMEUP', 10.000999999999896, None)
(0, 'REDGOAL', 1.5749999999999373, None)
(0, 'REDGOAL', 3.7089999999997025, None)
(0, 'REDGO

In [ ]:
################
# Small Oracle #
################

import random

def small_oracle(samples, num_sim_1, num_sim_2, iters=1000):
    '''
    samples:        Pre-run samples to draw from, list [Ball_1, Ball_2]
    num_sim_1:      Number of simulations for ball 1
    num_sim_2:      Number of simulations for ball 2
    samples:        Number of samples (each sample has num_sim_1 + 2 sims)
    Return tuple(# green wins, # red wins, avg total time)
    # green wins:   Number of samples where #G > #R + 0.5 * ties
    # red wins:     Number of samples where #G < #R + 0.5 * ties
    avg total time: Average total time for a sample
    '''
    g_wins = 0  # track green wins
    r_wins = 0  # track red wins
    total_time = 0  # total time

    for _ in range(iters):  # get samples
        # wins for a fixed sample
        sample_g_wins = 0
        sample_r_wins = 0
        sample_time = 0
        # simulations for ball 1
        for sim_1 in range(num_sim_1):
            table_id, outcome, time, ball = random.choice(samples[0])
            if outcome == 'GREENGOAL':
                sample_g_wins += 1
            if outcome == 'REDGOAL':
                sample_r_wins += 1
            sample_time += time
        # simulations for ball 2
        for sim_2 in range(num_sim_2):
            table_id, outcome, time, ball = random.choice(samples[1])
            if outcome == 'GREENGOAL':
                sample_g_wins += 1
            if outcome == 'REDGOAL':
                sample_r_wins += 1
            sample_time += time
        if sample_g_wins > sample_r_wins:
            g_wins += 1
        if sample_g_wins < sample_r_wins:
            r_wins += 1
        if sample_g_wins == sample_r_wins:
            g_wins += 0.5
            r_wins += 0.5
        total_time += sample_time
    return (g_wins, r_wins, total_time/iters)

In [ ]:
small_oracle([samples1, samples2], 5, 5, 10)